<a href="https://colab.research.google.com/github/JsuccessJ/dialect_transformer/blob/main/%EA%B2%BD%EC%83%81%EB%8F%841%EB%A7%8C_%ED%85%8C%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#설치 후 런타임 재실행해야함
!pip install accelerate>=0.20.1

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig,PreTrainedTokenizerFast
import torch


# Specify the directory containing the model and tokenizer files
model_dir = '/content/drive/MyDrive/hjse/jaesung/pretrained_경상도1/'

# Load the model and tokenizer
model = BartForConditionalGeneration.from_pretrained(model_dir)

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')



You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
# Input text in 사투리
satoori_text = '밥맛있노'

# Tokenize the input text
input_ids = tokenizer(satoori_text, return_tensors='pt').input_ids

# Generate the translation
translation_ids = model.generate(input_ids, max_length=50, num_beams=4, length_penalty=2.0, early_stopping=True)

# Decode the generated translation
translated_text = tokenizer.decode(translation_ids[0], skip_special_tokens=True)

# Print the translated text
print("번역 결과:", translated_text)

번역 결과: 밥맛있어


# test_data로 evaluate

In [ ]:
import nltk.translate.bleu_score as bleu
#라이브러리 총집합
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/hjse/jaesung/dataset_경상도1만개_test.csv')

In [ ]:
device = torch.device("cuda:0")

In [ ]:
# 1열(첫 번째 열)에서 가장 긴 문자열의 길이 찾기
max_length = data.iloc[:, 0].str.len().max()

# 가장 긴 문자열 찾기
longest_sentence = data.loc[data.iloc[:, 0].str.len() == max_length, data.columns[0]].values[0]

print("가장 긴 문장의 글자수:", max_length)
print("가장 긴 문장을 출력함:", longest_sentence)

가장 긴 문장의 글자수: 107
가장 긴 문장을 출력함: 이제 나 엄마도 잘 모르는 게 있겠지만 엄마가 밖에 나가서 내 가끔 집에서 이제 고기를 하다못해 아주 간단한 고기를 굽는다던가 그런 거 하다가 내가 태워 태워 먹은 것도 내가 몇 번 있어가지구


In [ ]:
import torch
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from transformers import EvalPrediction

# 평가함수 정의 (bleu score 출력)
def compute_metrics(p: EvalPrediction):
    preds = p.predictions
    labels = p.label_ids

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # BLEU 스코어 계산을 위해 두 개의 리스트로 만들기
    references = [[label.strip()] for label in decoded_labels]
    predictions = [pred.strip() for pred in decoded_preds]

    # BLEU 스코어 계산
    from nltk.translate.bleu_score import corpus_bleu
    bleu_score = corpus_bleu(references, predictions)

    return {"bleu_score": bleu_score}


# evalDataset 클래스 정의
class evalDataset(Dataset):
    def __init__(self, standard, dialect, tokenizer):
        self.standard = standard
        self.dialect = dialect
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.standard)

    def __getitem__(self, idx):
        standard_text = self.standard[idx]
        dialect_text = self.dialect[idx]

        # 토크나이징 및 패딩 수행
        inputs = self.tokenizer(
            text=dialect_text,
            padding="max_length",  # 필요한 경우 max_length 설정
            truncation=True,       # 필요한 경우 문장 자르기 설정
            return_tensors="pt",   # PyTorch 텐서로 반환
            max_length=130,
            )

        outputs = self.tokenizer(
            text=standard_text,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            max_length=130,
        )

        # 모델 입력을 포함하는 딕셔너리 반환
        sample = {
            'input_ids': inputs.input_ids.squeeze(),
            'attention_mask': inputs.attention_mask.squeeze(),
            'labels': outputs.input_ids.squeeze()
        }

        return sample


In [ ]:
# 테스트 데이터 전처리

# 1열(standard)과 2열(dialect)을 각각 features(X)와 labels(y)로 설정
x = data['standard']
y = data['dialect']

test_dataset = evalDataset(x, y, tokenizer)



In [ ]:
print(x)

0                            제일 힘이 세 보이니까 꾸는 꿈이 대통령이잖아.
1                  그러니까 열두 시에 가가지고 문 뚜들겨서 관장님 문 열어달라 해서
2                                  그러니까 운동이 너무 좋아서 계속 막
3                          그냥 나는 그냥 너무 좋아서 막 누구를 이기려고 막
4       오기가 생겨서 막 이렇게 하다 보니까 너는 대회에 나가야겠다 해가지고 대회를 나갔어.
                             ...                       
9995               또 속리산 거기가 이제 나 (()) 큰 산이거든 또 지리산 가서는
9996      거서 자고 왔어 왜냐면 힘이 올라가는 데 힘 다 빠져버려 내려올 수가 없는 거야.
9997                또 거서 한번 산에 산에 산장에서 함 자보 자는 것도 경험이고.
9998         그래 이제 이~ 다녔는데 고만 인제 힘이 빠져 버리니까 뭐~ 아무도 못 가.
9999                                못 한 못 한다고 힘으로 하는 건.
Name: standard, Length: 10000, dtype: object


In [ ]:
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments


args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/hjse/jaesung/pretrained_경상도1_newTest",
    predict_with_generate=True,

)


# Trainer 객체 생성 (훈련 부분 제거)
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    compute_metrics=compute_metrics,
    eval_dataset = test_dataset,

)

# 테스트 데이터를 평가 데이터셋으로 설정
test_dataset = evalDataset(x, y ,tokenizer)



In [ ]:
# 첫 번째 샘플 얻기
first_sample = test_dataset[0]
print(first_sample)

{'input_ids': tensor([17629, 18163,  1700, 11546, 16412, 14495, 15353,  9698, 29676, 15690,
        16223,   245,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3, 

In [ ]:

# 평가 수행
results = trainer.evaluate()


# 평가 결과 출력
print(results)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 0.03678014501929283, 'eval_bleu_score': 0.8650577487945508, 'eval_runtime': 271.2349, 'eval_samples_per_second': 36.868, 'eval_steps_per_second': 4.609}
